In [ ]:
import uproot
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np 
import matplotlib as mpl

from util import *
import var
import cut
import data
import hist

import importlib

from pyanalib import panda_helpers

In [ ]:
importlib.reload(var)
importlib.reload(cut)
importlib.reload(data)
importlib.reload(hist)
importlib.reload(panda_helpers)

In [ ]:
plt.rcParams.update({'font.size': 16})
mpl.rcParams['lines.linewidth'] = 1

dosave = True
savedir = "plots_2_12_24/cohlike-validation/"

In [ ]:
filedir = "/icarus/data/users/gputnam/DMCP2023G/mc-F/"
fnu_incoh = filedir + "F-MCNuPhase2_evt.df"

filedir = "/icarus/data/users/gputnam/DMCP2023G/mc-F/"
# fnu_coh = filedir + "FB-CohLike_nom-polaris-prod3_evt.df"
fnu_coh = filedir + "F-CohLike_nom_evt.df"

In [ ]:
nu_evtdf_coh = pd.read_hdf(fnu_coh, key="evt")
nu_mcdf_coh = pd.read_hdf(fnu_coh, key="mcnu")

In [ ]:
nu_evtdf_incoh = pd.read_hdf(fnu_incoh, key="evt")
nu_mcdf_incoh = pd.read_hdf(fnu_incoh, key="mcnu")

In [ ]:
nu_hdr_coh = pd.read_hdf(fnu_coh, key="hdr")
nu_hdr_incoh = pd.read_hdf(fnu_incoh, key="hdr")

In [ ]:
nu_hdr_coh

In [ ]:
NU_PER_POT = nu_mcdf_coh.shape[0] / nu_hdr_coh.pot.sum()

In [ ]:
NU_PER_POT

In [ ]:
NU_PER_POT_BASE = 1.0448039251186124e-16

In [ ]:
nu_mcdf_incoh["scale"] = 2e20 / nu_hdr_incoh.pot.sum()
nu_evtdf_incoh["scale"] = 2e20 / nu_hdr_incoh.pot.sum()

if NU_PER_POT / NU_PER_POT_BASE > 0.8: # check POT normalized correctly
    nu_mcdf_coh["scale"] = 2e20 / nu_hdr_coh.pot.sum()
    nu_evtdf_coh["scale"] = 2e20 / nu_hdr_coh.pot.sum()    
else:
    nu_mcdf_coh["scale"] = 2e20/(nu_mcdf_coh.shape[0]/NU_PER_POT_BASE)
    nu_evtdf_coh["scale"] = 2e20/(nu_mcdf_coh.shape[0]/NU_PER_POT_BASE)

In [ ]:
def cohlike(df):
    nmu = df.iscc & (np.abs(df.pdg) == 14)
    npi = df.npi
    ns = df.nsm + df.nsp

    is_coh_like = (nmu + npi + ns >= 2) & (df.max_proton_ke < 0.05)

    is_coh_like = is_coh_like & InFV(df.position, 50)

    return is_coh_like

In [ ]:
cohlike(nu_mcdf_coh).groupby(level=[0,1]).any().sum() / (cohlike(nu_mcdf_coh).groupby(level=[0,1]).any().sum() | True).sum()

In [ ]:
nu_mcdf_coh["cohlike"] = True
nu_mcdf_incoh["cohlike"] = cohlike(nu_mcdf_incoh).groupby(level=[0,1]).any()

In [ ]:
nu_evtdf_coh["cohlike"] = True
nu_evtdf_incoh["cohlike"] = cohlike(nu_mcdf_incoh).groupby(level=[0,1]).any()

In [ ]:
def when(df):
    return SlcInFV(df.position) & df.cohlike & cohlike(df)

In [ ]:
def whenevt(df):
    return SlcInFV(df.slc.vertex) & df.cohlike & ~np.isnan(df.slc.truth.E)

In [ ]:
coh_err_wgt = nu_evtdf_coh.scale.iloc[0]
incoh_err_wgt = nu_evtdf_incoh.scale.iloc[0]

In [ ]:
bins = np.linspace(0, 5, 11)
centers = (bins[:-1] + bins[1:]) / 2

Ncoh, _ = np.histogram(nu_mcdf_coh.E[when(nu_mcdf_coh)], 
                       weights=nu_mcdf_coh.scale[when(nu_mcdf_coh)], bins=bins)
Nincoh, _ = np.histogram(nu_mcdf_incoh.E[when(nu_mcdf_incoh)], 
                         weights=nu_mcdf_incoh.scale[when(nu_mcdf_incoh)], bins=bins)

fig, (p0, p1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3,1]}, sharex=True, figsize=(7, 6.4))
fig.subplots_adjust(hspace=0.03)

p0.errorbar(centers, Ncoh, np.sqrt(Ncoh*coh_err_wgt),
            linestyle="none", marker=".", label="Coh-Like")
p0.errorbar(centers, Nincoh, np.sqrt(Nincoh*incoh_err_wgt),
            linestyle="none", marker=".", color="black", label="MCNuPhase2\n(Nom.)")
p0.legend(title="Production")

ratio = Ncoh / Nincoh
ratio_err = np.sqrt(coh_err_wgt*ratio**2/Ncoh + incoh_err_wgt*ratio**2/Nincoh)

p1.errorbar(centers, ratio, ratio_err, marker=".", linestyle="none")
p1.axhline([1], color="r", linestyle=":")
p1.set_ylim([0.95, 1.05])

p1.set_xlabel("Neutrino Energy [GeV]")
p0.set_ylabel("Events / $2 \\times 10^{20}$ POT")
p1.set_ylabel("Coh-like / Nom")

if dosave:
    plt.savefig(savedir + "nuE.pdf", bbox_inches="tight")

In [ ]:
bins = np.linspace(0, 500, 11)
centers = (bins[:-1] + bins[1:]) / 2

Ncoh, _ = np.histogram(nu_evtdf_coh.trunk.trk.len[whenevt(nu_evtdf_coh)], 
                       weights=nu_evtdf_coh.scale[whenevt(nu_evtdf_coh)], bins=bins)
Nincoh, _ = np.histogram(nu_evtdf_incoh.trunk.trk.len[whenevt(nu_evtdf_incoh)], 
                         weights=nu_evtdf_incoh.scale[whenevt(nu_evtdf_incoh)], bins=bins)

fig, (p0, p1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3,1]}, sharex=True, figsize=(7, 6.4))
fig.subplots_adjust(hspace=0.03)

p0.errorbar(centers, Ncoh, np.sqrt(Ncoh*coh_err_wgt),
            linestyle="none", marker=".", label="Coh-Like")
p0.errorbar(centers, Nincoh, np.sqrt(Nincoh*incoh_err_wgt),
            linestyle="none", marker=".", color="black", label="MCNuPhase2\n(Nom.)")
p0.legend(title="Production")

ratio = Ncoh / Nincoh
ratio_err = np.sqrt(coh_err_wgt*ratio**2/Ncoh + incoh_err_wgt*ratio**2/Nincoh)

p1.errorbar(centers, ratio, ratio_err, marker=".", linestyle="none")
p1.axhline([1], color="r", linestyle=":")
p1.set_ylim([0.9, 1.1])

p1.set_xlabel("Length of Track Closest to Vertex [cm]")
p0.set_ylabel("Events / $2 \\times 10^{20}$ POT")
p1.set_ylabel("Coh-like / Nom")

if dosave:
    plt.savefig(savedir + "trunk_track_length.pdf", bbox_inches="tight")

In [ ]:
bins = np.linspace(0, 30, 11)
centers = (bins[:-1] + bins[1:]) / 2

Ncoh, _ = np.histogram(nu_evtdf_coh.trunk.trk.chi2pid.I2.chi2u[whenevt(nu_evtdf_coh)], 
                       weights=nu_evtdf_coh.scale[whenevt(nu_evtdf_coh)], bins=bins)
Nincoh, _ = np.histogram(nu_evtdf_incoh.trunk.trk.chi2pid.I2.chi2u[whenevt(nu_evtdf_incoh)], 
                         weights=nu_evtdf_incoh.scale[whenevt(nu_evtdf_incoh)], bins=bins)

fig, (p0, p1) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [3,1]}, sharex=True, figsize=(7, 6.4))
fig.subplots_adjust(hspace=0.03)

p0.errorbar(centers, Ncoh, np.sqrt(Ncoh*coh_err_wgt),
            linestyle="none", marker=".", label="Coh-Like")
p0.errorbar(centers, Nincoh, np.sqrt(Nincoh*incoh_err_wgt),
            linestyle="none", marker=".", color="black", label="MCNuPhase2\n(Nom.)")
p0.legend(title="Production")

ratio = Ncoh / Nincoh
ratio_err = np.sqrt(coh_err_wgt*ratio**2/Ncoh + incoh_err_wgt*ratio**2/Nincoh)

p1.errorbar(centers, ratio, ratio_err, marker=".", linestyle="none")
p1.axhline([1], color="r", linestyle=":")
p1.set_ylim([0.9, 1.1])

p1.set_xlabel("$\\chi^2_\\mu$")
p0.set_ylabel("Events / $2 \\times 10^{20}$ POT")
p1.set_ylabel("Coh-like / Nom")